# Step 2: Calibrated Street Network Generation

**Generate new pedestrian networks calibrated to reference city metrics**

This notebook:
1. Loads reference metrics from Step 1
2. Generates new pedestrian networks matching target metrics
3. Validates generated networks against reference distributions
4. Exports generated networks for urban analysis

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from pathlib import Path
from collections import Counter
import math
import pickle
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import unary_union

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Configuration

In [ ]:
WINDOW_SIZE_M = 500  # 500×500m window
MIN_SEGMENT_LENGTH = 5.0  # Filter segments < 5m

# Create output directories
Path("outputs/generated").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/visualizations").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/networks").mkdir(parents=True, exist_ok=True)

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Min segment length: {MIN_SEGMENT_LENGTH}m")
print("✓ Output directories created")

## Load Reference Data from Step 1

In [ ]:
# Load reference city data
with open('outputs/data/reference_cities_data.pkl', 'rb') as f:
    reference_data = pickle.load(f)

print("✓ Loaded reference data from Step 1")
print(f"\nReference cities: {list(reference_data.keys())}")

# Print summary
print("\n" + "="*70)
print("REFERENCE METRICS SUMMARY")
print("="*70)

for city_key, data in reference_data.items():
    print(f"\n{city_key.upper()}:")
    print(f"  Network: {data['graph'].number_of_nodes()} nodes, {data['graph'].number_of_edges()} edges")
    print(f"  Node density: {data['morphology']['node_density']:.1f} nodes/km²")
    print(f"  Avg degree: {data['morphology']['avg_degree']:.2f}")
    print(f"  Avg segment length: {data['morphology']['avg_segment_length']:.1f}m")
    print(f"  Buildings: {len(data['buildings'])} (coverage: {data['building_metrics']['building_coverage_ratio']:.3f})")
    print(f"  Intelligibility: {data['syntax']['intelligibility']:.3f}")

## Network Generation Functions

In [ ]:
def generate_grid_network(width, height, spacing=50):
    """
    Generate a basic grid network as baseline.
    
    Args:
        width: Width in meters
        height: Height in meters
        spacing: Grid spacing in meters
    
    Returns:
        NetworkX graph
    """
    G = nx.Graph()
    
    # Create grid nodes
    node_id = 0
    node_positions = {}
    
    for x in range(0, width + 1, spacing):
        for y in range(0, height + 1, spacing):
            G.add_node(node_id, x=x, y=y)
            node_positions[node_id] = (x, y)
            node_id += 1
    
    # Create edges
    nodes_per_row = (height // spacing) + 1
    
    for node in G.nodes():
        x, y = G.nodes[node]['x'], G.nodes[node]['y']
        
        # Right neighbor
        if x + spacing <= width:
            neighbor = node + nodes_per_row
            if neighbor in G.nodes():
                G.add_edge(node, neighbor, length=spacing)
        
        # Up neighbor
        if y + spacing <= height:
            neighbor = node + 1
            if neighbor in G.nodes():
                G.add_edge(node, neighbor, length=spacing)
    
    return G, node_positions


def add_noise_to_network(G, pos, noise_level=5.0):
    """
    Add random perturbations to node positions.
    
    Args:
        G: NetworkX graph
        pos: Node positions dict
        noise_level: Max displacement in meters
    
    Returns:
        Updated positions dict
    """
    new_pos = {}
    
    for node, (x, y) in pos.items():
        dx = np.random.uniform(-noise_level, noise_level)
        dy = np.random.uniform(-noise_level, noise_level)
        
        # Clamp to window
        new_x = max(0, min(WINDOW_SIZE_M, x + dx))
        new_y = max(0, min(WINDOW_SIZE_M, y + dy))
        
        new_pos[node] = (new_x, new_y)
        G.nodes[node]['x'] = new_x
        G.nodes[node]['y'] = new_y
    
    # Update edge lengths
    for u, v in G.edges():
        x1, y1 = new_pos[u]
        x2, y2 = new_pos[v]
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        G[u][v]['length'] = length
    
    return new_pos


def randomly_remove_edges(G, removal_rate=0.2):
    """
    Randomly remove edges to create organic irregularity.
    
    Args:
        G: NetworkX graph
        removal_rate: Fraction of edges to remove (0-1)
    
    Returns:
        Modified graph
    """
    edges = list(G.edges())
    num_to_remove = int(len(edges) * removal_rate)
    
    edges_to_remove = np.random.choice(len(edges), num_to_remove, replace=False)
    
    for idx in edges_to_remove:
        u, v = edges[idx]
        G.remove_edge(u, v)
    
    # Remove isolated nodes
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    
    return G


print("✓ Network generation functions defined")

## Generate Test Network

In [ ]:
# Generate a test network
print("Generating test network...\n")

# Start with grid
G_test, pos_test = generate_grid_network(WINDOW_SIZE_M, WINDOW_SIZE_M, spacing=50)
print(f"Initial grid: {G_test.number_of_nodes()} nodes, {G_test.number_of_edges()} edges")

# Add noise
pos_test = add_noise_to_network(G_test, pos_test, noise_level=10.0)
print(f"After noise: {G_test.number_of_nodes()} nodes, {G_test.number_of_edges()} edges")

# Remove some edges
G_test = randomly_remove_edges(G_test, removal_rate=0.15)
print(f"After edge removal: {G_test.number_of_nodes()} nodes, {G_test.number_of_edges()} edges")

# Update positions after node removal
pos_test = {n: (G_test.nodes[n]['x'], G_test.nodes[n]['y']) for n in G_test.nodes()}

print("\n✓ Test network generated")

## Visualize Generated Network

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Window boundary
ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                       fill=False, edgecolor='black', linestyle='-', linewidth=2))

# Draw edges
for u, v in G_test.edges():
    x = [pos_test[u][0], pos_test[v][0]]
    y = [pos_test[u][1], pos_test[v][1]]
    ax.plot(x, y, color='blue', linewidth=1.5, alpha=0.7, zorder=1)

# Draw nodes
degrees = dict(G_test.degree())
max_degree = max(degrees.values()) if degrees else 1

for node in G_test.nodes():
    degree = degrees[node]
    color_val = degree / max_degree
    node_color = plt.cm.RdYlBu_r(color_val)
    ax.scatter(pos_test[node][0], pos_test[node][1], s=40, c=[node_color],
              zorder=2, edgecolors='black', linewidths=0.5)

ax.set_xlim(-20, WINDOW_SIZE_M + 20)
ax.set_ylim(-20, WINDOW_SIZE_M + 20)
ax.set_aspect('equal')
ax.set_title(f'Generated Test Network\n{G_test.number_of_nodes()} nodes, {G_test.number_of_edges()} edges',
            fontsize=14, fontweight='bold')
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/generated/visualizations/B0_test_network.svg', format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/generated/visualizations/B0_test_network.svg")

plt.show()

## Compute Metrics for Generated Network

In [ ]:
# Import metric functions from Step 1
def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    bearing = bearing % 180
    return bearing


def compute_morphology_metrics(G, pos):
    """Compute morphology metrics."""
    metrics = {}
    
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2
    
    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0
    
    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0
    
    lengths = []
    for u, v, data in G.edges(data=True):
        length = data.get('length', 0)
        lengths.append(length)
    
    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0
    
    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)
    
    if bearings:
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))
    
    return metrics


# Compute metrics
test_metrics = compute_morphology_metrics(G_test, pos_test)

print("Generated Network Metrics:")
print(f"  Node density: {test_metrics['node_density']:.1f} nodes/km²")
print(f"  Avg degree: {test_metrics['avg_degree']:.2f}")
print(f"  Dead-end ratio: {test_metrics['dead_end_ratio']:.3f}")
print(f"  Avg segment length: {test_metrics['avg_segment_length']:.1f}m")
print(f"  Degree distribution: {test_metrics['degree_distribution']}")

## Compare to Reference Cities

In [ ]:
# Create comparison table
comparison_data = []

# Add reference cities
for city_key, data in reference_data.items():
    comparison_data.append({
        'Network': city_key.upper(),
        'Type': 'Reference',
        'Nodes': data['graph'].number_of_nodes(),
        'Edges': data['graph'].number_of_edges(),
        'Node Density': f"{data['morphology']['node_density']:.1f}",
        'Avg Degree': f"{data['morphology']['avg_degree']:.2f}",
        'Avg Seg Length': f"{data['morphology']['avg_segment_length']:.1f}m"
    })

# Add generated network
comparison_data.append({
    'Network': 'GENERATED',
    'Type': 'Generated',
    'Nodes': G_test.number_of_nodes(),
    'Edges': G_test.number_of_edges(),
    'Node Density': f"{test_metrics['node_density']:.1f}",
    'Avg Degree': f"{test_metrics['avg_degree']:.2f}",
    'Avg Seg Length': f"{test_metrics['avg_segment_length']:.1f}m"
})

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "="*90)
print(" "*30 + "NETWORK COMPARISON")
print("="*90)
print(df_comparison.to_string(index=False))
print("="*90)

## Next Steps

This is a basic test network generation. Future improvements:

1. **Calibrated Generation**: Match specific reference city metrics
2. **Advanced Algorithms**: Use graph grammar, L-systems, or evolutionary algorithms
3. **Building Integration**: Generate building footprints matching reference distributions
4. **Space Syntax Optimization**: Optimize for intelligibility and integration
5. **Multi-objective Calibration**: Match multiple metrics simultaneously
6. **Validation**: Statistical tests comparing generated vs reference distributions